# 第七节 实战召回优化——重排序与多路召回

上一节我们讲了相似度优化的优化技巧，这一节内容我们将在代码层面实现优化召回率的其中一种方式——重排序与多路召回。

## 提升指标的优化策略之重排序与多路召回

\*重排序\*\*这个真的是个好东西！我们现在基本上都是先粗召回个几十篇，然后用个轻量级的重排模型筛选。这里有个坑要注意——重排模型最好用领域内的数据微调一下，通用模型在特定领域表现经常拉胯。

最后说说**多路召回**，这个策略我们现在用得特别多。打个比方，就像是撒网捕鱼，多撒几个网总比一个网捞得多。我们通常会配置 3-4 路不同的召回器，有的负责精确匹配，有的负责语义相似，最后再统一排序。虽然计算开销大了点，但效果是真的好。

那么如何从**代码层面实现**呢？

## reranker 重排序

我得说，重排序这个东西刚开始接触的时候，真没觉得有多重要。直到有一次，我们的**系统明明召回了正确答案，但用户还是抱怨说找不到想要的信息——原来是正确答案排在第四五位，用户根本没耐心往下看**。这才让我意识到，光召回还不够，排序才是王道。

记得我第一次用重排序是在一个法律文书检索的项目上。当时的情况是这样的：用户搜索"合同违约赔偿"，系统能召回十几个相关条款，但最相关的往往埋在中间。我们试了下 bge-reranker，效果那叫一个惊艳——最相关的条款直接顶到了第一位。从那以后，两阶段检索基本成了我们的标配。

说到 LazyLLM 的 Reranker 组件，用起来确实挺顺手的。我个人比较喜欢用本地的 bge-reranker-large，虽然启动慢了点，但效果稳定，而且不用担心 API 调用限制。在线模型我也试过，qwen 的重排序服务响应速度快，适合对延迟要求高的场景，但得注意控制成本——我有个同事没设置好调用频率，一个月的 API 费用差点把老板吓着。

这里有个细节挺重要的——`topk`的设置。我的经验是，初召回的时候可以放宽一点，比如召回 top 20 或 30，然后重排后只取 top 3 或 5。为啥这么做？因为初召回阶段用的是快速但粗糙的方法，难免会漏掉一些好结果。多召回一些，给重排序模型更多选择的空间，最终效果会更好。

**余弦相似度召回的结果顺序基本是随机的，或者按文档在数据库里的存储顺序**。用户问"猴面包树有哪些功效"，结果第一条是假苹婆，第三条才是猴面包树，这体验能好吗？重排序之后，猴面包树相关的内容直接排到第一位，这才是用户想要的。

关于**召回文档的平均倒数排名**（Mean Reciprocal Rank, MRR）这个指标，我想多说两句。很多人只关注召回率，觉得只要答案在结果里就行。但实际上，MRR 可能更重要——它**衡量的是正确答案的排序位置**。不用重排序的话，top 5 的 MRR 只有 0.44，意味着正确答案平均排在第二三位。用了重排序，MRR 飙到 0.97，基本上正确答案都在第一位了。

还有个实践中的小技巧：**重排序模型最好针对你的业务场景做微调**。通用的重排序模型在特定领域可能表现一般。我们之前在医疗领域的项目，用通用模型效果就不太理想，后来用领域内的 QA 对收集了几千条样本微调了一下，效果提升明显。

最后想说的是，重排序虽然增加了一步计算，但这个开销是值得的。特别是在用户体验要求高的场景下，宁可多花点计算资源，也要把最相关的结果排在前面。毕竟，**用户的耐心是有限的，他们不会翻到第二页去找答案**。

我们实现这样一个例子,对上述 cosine 和 bm25 检索的结果取精排后的文档：

In [ ]:
pip install lazyllm
export QWEN_API_KEY='sk-这里填写你的key'

In [ ]:
from lazyllm import Document, Retriever, OnlineEmbeddingModule, Reranker
import os

# 定义 embedding 模型
embedding_model = OnlineEmbeddingModule(
    source="qwen",
    api_key=os.getenv("QWEN_API_KEY"),
)

rerank_model = OnlineEmbeddingModule(
    source="qwen",
    api_key=os.getenv("QWEN_API_KEY"),
    type="rerank",
)


docs = Document("./data_kb", embed=embedding_model)
docs.create_node_group(name="block", transform=(lambda d: d.split("\n")))

# 定义检索器
retriever = Retriever(docs, group_name="block", similarity="cosine", topk=3)

# 定义重排器
# 指定 reranker1 的输出为字典，包含 content, embedding 和 metadata 关键字
# reranker = Reranker('ModuleReranker', model=online_rerank, topk=3, output_format='dict')
# 指定 reranker2 的输出为字符串，并进行串联，未进行串联时输出为字符串列表
reranker = Reranker(
    "ModuleReranker", model=rerank_model, topk=3, output_format="content", join=True
)

# 执行推理
query = "猴面包树有哪些功效？"
result1 = retriever(query=query)
result2 = reranker(result1, query=query)

print("余弦相似度召回结果：")
print("\n\n".join([res.get_content() for res in result1]))
print("开源重排序模型结果：")
print(result2)

下表是针对查询“猴面包树有哪些功效？”的召回结果，注意到余弦相似度召回结果（第一列）的顺序并不包含相似度排序信息，通常是按照其在节点组内的相对位置进行返回，而进行一次重排序得到的结果如第二列所示，可见重排序后与查询更相似的文档被排在了在更前面的位置。

|     | **余弦相似度召回结果**                                                                                                                                                            | 重排结果                                                                                                                                                                          |
| --- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 1   | 假苹婆 （学名：" "），别称七姐果、鸡冠皮、鸡冠木、山羊角、红郎伞、赛苹婆'及山木棉等，为梧桐科苹婆属半落叶乔木植物...                                                              | 猴面包树（学名：），又名-{zh-hant:猴面包树;zh-hans:猢狲树;zh-tw:猴面包树}-、非洲猴面包树、酸瓠树、猴树、旅人树、死老鼠树，是一种锦葵科猴面包树属的大型落叶乔木，原产于热带非洲... |
| 2   | 黄蝉（学名：'）别称为硬枝黄蝉、夹竹桃叶黄蝉、小花黄蝉、丛立黄蝉及黄兰蝉。为夹竹桃科黄蝉属植物...                                                                                  | 黄蝉（学名：'）别称为硬枝黄蝉、夹竹桃叶黄蝉、小花黄蝉、丛立黄蝉及黄兰蝉。为夹竹桃科黄蝉属植物...                                                                                  |
| 3   | 猴面包树（学名：），又名-{zh-hant:猴面包树;zh-hans:猢狲树;zh-tw:猴面包树}-、非洲猴面包树、酸瓠树、猴树、旅人树、死老鼠树，是一种锦葵科猴面包树属的大型落叶乔木，原产于热带非洲... | 假苹婆 （学名：" "），别称七姐果、鸡冠皮、鸡冠木、山羊角、红郎伞、赛苹婆'及山木棉等，为梧桐科苹婆属半落叶乔木植物...                                                              |

## 综合上述所有优化的 RAG 系统

综合上述策略，我们可以得到**多路召回 RAG** ，即设置多个检索器在不同粒度进行检索或使用不同的相似度计算方法进行文档召回，再对所有检索器召回的文档进行排序得到最终上下文的方法。其流程如下图所示，这是一种比较标准的**多路检索 RAG 架构**，具有比朴素 RAG 更好的检索效果。

![image.png](https://docs.lazyllm.ai/zh-cn/stable/Tutorial/7_images/img7.png)

上述 RAG 流程的 LazyLLM 实现如下方所示：

In [ ]:
import lazyllm
import os

# 定义嵌入模型和重排序模型
embedding_model = lazyllm.OnlineEmbeddingModule(
    source="qwen",
    api_key=os.getenv("QWEN_API_KEY"),
)

rerank_model = lazyllm.OnlineEmbeddingModule(
    source="qwen",
    api_key=os.getenv("QWEN_API_KEY"),
    type="rerank",
)

docs = lazyllm.Document("./data_kb", embed=embedding_model)
docs.create_node_group(name="block", transform=(lambda d: d.split("\n")))

# 定义检索器
retriever1 = lazyllm.Retriever(
    docs, group_name="CoarseChunk", similarity="cosine", topk=3
)
retriever2 = lazyllm.Retriever(
    docs, group_name="block", similarity="bm25_chinese", topk=3
)

# 定义重排器
reranker = lazyllm.Reranker("ModuleReranker", model=rerank_model, topk=3)

# 定义大模型
llm = lazyllm.OnlineChatModule(
    source="qwen",
    model="qwen-plus-latest",
    api_key=os.getenv("QWEN_API_KEY"),
)

# prompt 设计
prompt = "你是一个友好的 AI 问答助手，你需要根据给定的上下文和问题提供答案。\
          根据以下资料回答问题：\
          {context_str} \n "
llm.prompt(lazyllm.ChatPrompter(instruction=prompt, extra_keys=["context_str"]))

# 执行推理
query = "2008年有哪些赛事？"
result1 = retriever1(query=query)
result2 = retriever2(query=query)
result = reranker(result1 + result2, query=query)

# 将query和召回节点中的内容组成dict，作为大模型的输入
res = llm(
    {"query": query, "context_str": "".join([node.get_content() for node in result])}
)

print(f"Answer: {res}")

输出结果如下：

```bash
Answer: 2008年举行的赛事包括：

1. **2008年夏季奥林匹克运动会男子100米跑赛事**
   - 时间：2008年8月15日及8月16日
   - 地点：北京国家体育馆
   - 亮点：牙买加运动员尤塞恩·博尔特（Usain Bolt）在决赛中以9秒69的成绩打破世界纪录，夺得金牌，这是他首次获得奥运男子100米金牌。

2. **2008年北京奥运会游泳比赛**
   - 荷兰女子游泳运动员拉诺米·克罗莫维焦约（Ranomi Kromowidjojo）参加4×100米自由泳接力赛，与队友因赫·德克、马伦·费尔德海斯和费姆克·海姆斯凯克合作，赢得金牌，成绩为3分33秒76，接近她们保持的世界纪录。

3. **2008年世界游泳锦标赛（长池）**
   - 时间：2008年
   - 地点：荷兰燕豪芬（Eindhoven）
   - 荷兰队在4×100米自由泳接力项目中赢得金牌，并刷新世界纪录（3分33秒62）。

4. **2008年国际泳联世界游泳锦标赛（短池）**
   - 时间：2008年
   - 地点：英国曼彻斯特
   - 拉诺米·克罗莫维焦约与队友赢得4×200米自由式接力赛冠军，并再次打破世界纪录。

5. **香港羽毛球公开赛（又称香港羽毛球超级赛）**
   - 时间：通常在每年11月举行（2008年也不例外）
   - 地点：红磡香港体育馆（自2011年起为主场馆，但2008年可能仍在湾仔伊利沙伯体育馆举行）
   - 赛事级别：世界羽联超级系列赛的一站，吸引世界顶级选手参赛。

这些是资料中提到的2008年举行的重要赛事。
```

如何把服务上线呢？

In [ ]:
import lazyllm
from lazyllm import bind
import os

# 初始化文档对象，指定数据目录和嵌入模型
# 使用Qwen的在线嵌入模型进行文档向量化
docs = lazyllm.Document(
    "./data_kb",
    embed=lazyllm.OnlineEmbeddingModule(
        source="qwen",
        api_key=os.getenv("QWEN_API_KEY"),
    ),
)

# 创建名为"block"的节点组，使用换行符分割文档内容
# 这样可以将文档按行分割成多个节点，便于后续检索
docs.create_node_group(name="block", transform=(lambda d: d.split("\n")))

# 定义提示词模板，指导模型如何根据文档内容回答问题
prompt = "你是一个知识问答助手，请通过给定的文档回答用户问题。"

# 构建处理管道(pipeline)
with lazyllm.pipeline() as ppl:
    # 并行处理模块，用于同时执行多个检索器
    with lazyllm.parallel().sum as ppl.prl:
        # CoarseChunk是LazyLLM默认提供的大小为1024的分块名
        # 使用BM25算法的中文检索器，从"block"节点组中检索top3相关文档
        ppl.prl.retriever1 = lazyllm.Retriever(
            doc=docs, group_name="block", similarity="bm25_chinese", topk=3
        )
        # 使用余弦相似度的检索器，从"block"节点组中检索top3相关文档
        ppl.prl.retriever2 = lazyllm.Retriever(
            doc=docs, group_name="block", similarity="cosine", topk=3
        )

    # 重排序模块，使用在线重排序模型对检索结果进行重新排序
    # 通过ModuleReranker和Qwen的在线重排序模型，选择最相关的top3文档
    ppl.reranker = lazyllm.Reranker(
        name="ModuleReranker",
        model=lazyllm.OnlineEmbeddingModule(
            source="qwen",
            api_key=os.getenv("QWEN_API_KEY"),
            type="rerank",
        ),
        topk=3,
    ) | bind(query=ppl.input)

    # 格式化模块，将检索到的节点内容和查询组合成字典格式
    # 为大模型准备输入数据，包含上下文和用户查询
    ppl.formatter = (
        lambda nodes, query: dict(
            context_str="".join([node.get_content() for node in nodes]),
            query=query,
        )
    ) | bind(query=ppl.input)

    # 大模型模块，使用Qwen-plus模型进行问答
    # 配置提示词模板和额外的上下文键
    ppl.llm = lazyllm.OnlineChatModule(
        source="qwen",
        model="qwen-plus-latest",
        api_key=os.getenv("QWEN_API_KEY"),
    ).prompt(lazyllm.ChatPrompter(instruction=prompt, extra_keys=["context_str"]))


# 启动Web服务模块，将处理管道部署为Web服务
# 指定端口号为23491，并等待服务启动完成
webpage = lazyllm.WebModule(ppl, port=23491).start().wait()

以下是输入“2008 年有哪些赛事？”的回答记录：

![image](../assets/image-20250907193157-bp85qst.png "2008年有哪些赛事？")